In [ ]:
!pip install bertopic[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [ ]:
from typing import List, Dict, Any, Optional, Tuple
import numpy as np, pandas as pd, math
from tqdm import tqdm
from umap import UMAP
import openai
import nltk
import time
import os
import re
import unicodedata
import copy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import OpenAI
from openai import OpenAI as OpenAIClient
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from collections import Counter
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, fcluster
from itertools import product

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


### 토크나이저 불용어 정의

In [ ]:
bertopic_stopwords = [
    # --- 특허 형식어 및 관용구 ---








]

In [ ]:
open_ai_key=userdata.get('OPENAI_API_KEY')

### 전처리 코드

In [ ]:
# --- 최초 1회만 필요 ---
nltk.download("punkt"); nltk.download("stopwords")
nltk.download('punkt_tab')
# =========================
# 불용어 세트 (언어+도메인)
# =========================

# 도메인(저신호 구조·형상/부품 일반명사)
DOMAIN_STOP = {
    "refrigerator","body","box","lid","plate","rack","space",
    "assembly","assemblies","housing","housings","case","cases","compartment","compartments",
    "chamber","chambers","container","containers","shelf","shelves","drawer","drawers",
    "bin","bins","basket","baskets","tray","trays","partition","partitions","panel","panels",
    "cover","covers","frame","frames","bracket","brackets","guide","guides","groove","grooves",
    "opening","openings","closure","closures","hinge","hinges","handle","handles","support","supports",
    "member","element","component","structure","mechanism","unit","modules","apparatus","device",
    "storage","control","controller","controllers","position",

    # 배관/유로 주변 저신호
    "evaporator","evaporators","condenser","condensers","compressor","compressors",
    "fan","fans","blower","blowers","duct","ducts","nozzle","nozzles",
    "tube","tubes","pipe","pipes","conduit","conduits","manifold","manifolds",
    "aperture","apertures","edge","edges","corner","corners","sidewall","sidewalls",

    # 의미 희석 일반어
    "composition","agent","parts","weight","mass","type","sample"
}

ALL_STOP = DOMAIN_STOP

# =========================
# 정규표현식 (라벨/단위/형식 제거)
# =========================
_RE_PATENT_NUM   = re.compile(r'\b(?:us|wo|kr|ep|jp|cn)\s*\d+[a-z]*\d*\b', re.I)
_RE_CLAIM_EN     = re.compile(r'\b(claim|claims)\s*\d+\b', re.I)
_RE_CLAIM_KO     = re.compile(r'제\s*\d+\s*항')
_RE_FIG          = re.compile(r'(?:fig\.?|figure|도)\s*\d+[a-z]*', re.I)
_RE_TABLE        = re.compile(r'table\s*\d+', re.I)
_RE_EXAMPLE_HDR  = re.compile(r'\b(ex|example|examples)\s*\d+[a-z]*\b', re.I)

# 국문 섹션 헤더(샘플 반영)
_RE_SECTION_HDR_KO = re.compile(
    r'(발명의\s*분야|기술\s*분야|배경\s*기술|발명의\s*요약|과제|해결수단|효과|도면의\s*간단한\s*설명|'
    r'발명의\s*상세한\s*설명|목적)\b'
)

# “것을 특징으로 하는/한다/한다.” 관용구 컷
_RE_FEATURE_PHRASE = re.compile(r'것을\s*특징으로\s*하는(?:\s*것)?', re.I)

# 참조라벨(12a, b1, 3v 등)
_RE_REF_LABEL    = re.compile(r'\b(?:[a-z]{1,3}\d{1,4}|\d{1,4}[a-z]{1,3})\b', re.I)

# 수치-단위/범위/백분표기
_RE_NUM_UNIT     = re.compile(r'\b-?\d+(?:\.\d+)?\s*(?:nm|mm|cm|μm|um|kg|g|mg|l|ml|rpm|hz|khz|mhz|ghz|db|v|kv|ma|a|w|kw|wh|ppm|bar|psi|kpa|pa|°c|℃|°f|℉|%)\b', re.I)
_RE_WT_VOL_MOL   = re.compile(r'\b(?:wt|vol|mol)\.?\s*%\b', re.I)
_RE_RANGE        = re.compile(r'\b-?\d+(?:\.\d+)?\s*(?:~|–|—|-|to)\s*-?\d+(?:\.\d+)?\s*(?:%|ppm)?\b', re.I)

# 특수기호 + (10), (12a)류
_RE_SYMBOLS      = re.compile(r'[±≤≥≦≧≈∼~×™®©•·…§†º‡°‰℃℉µΩΔαβγμΩ※]')
_RE_PAREN_NUM    = re.compile(r'\(\s*\d{1,3}[a-z]?\s*\)')

_RE_DIGIT = re.compile(r'\d+')

_RE_CLAIM_KO_INLINE = re.compile(
    r'(?:청구항\s*\d+(?:\s*(?:[,~–—\-]|내지|및|또는)\s*\d+)*)'
    r'|(?:제\s*\d+\s*항(?:\s*(?:[,~–—\-]|내지|및|또는)\s*제?\s*\d+\s*항)*)',
    re.I
)
_RE_INLINE_FORMULA = re.compile(
    r'(?:\|[^\|>]+>\s*(?:=|\+|-|\*|/)?\s*)+'   # |ψ>, |0>, |1>, |+> 등 브라켓 ket 기호
    r'|(?:[α-ωΑ-Ω]\s*(?:=|\+|-|\*|/)?\s*)+'    # 그리스 문자 α, β, ψ, Ω 등
    r'|(?:[A-Za-z]\s*=\s*[^,;\n]+)',           # 단일 문자 = 식 (예: x = y+z)
    re.UNICODE
)

# 문제되는 빅그램(원하면 추가)
_RE_PHRASES = [
    re.compile(r'\bheat\s+exchanger\b', re.I),
    re.compile(r'\bkimchi\s+refrigerator\b', re.I),
]

def _normalize(text: str) -> str:
    t = unicodedata.normalize("NFKC", text).lower()
    # 하이픈/슬래시 제외 특수문자 제거
    t = re.sub(r"[^\w\s\-/]", " ", t)
    return t

def simple_preprocess_for_embedding(texts: list[str]) -> list[str]:
    """
    임베딩 전처리(업데이트):
      - 한국어 특허 보일러플레이트/형식어, 위치어, 조작 일반어 대폭 확장
      - 국문 섹션 헤더/관용구(‘것을 특징으로 하는’) 제거
      - 특허번호/도면/라벨/단위/범위/특수기호 제거
    """
    out = []
    for text in texts:
        if not isinstance(text, str) or not text.strip():
            out.append("")
            continue

        t = _normalize(text)

        # 형식/헤더/참조
        t = _RE_PATENT_NUM.sub(" ", t)
        t = _RE_CLAIM_EN.sub(" ", t); t = _RE_CLAIM_KO.sub(" ", t)
        t = _RE_FIG.sub(" ", t); t = _RE_TABLE.sub(" ", t); t = _RE_EXAMPLE_HDR.sub(" ", t)
        t = _RE_SECTION_HDR_KO.sub(" ", t)
        t = _RE_FEATURE_PHRASE.sub(" ", t)

        # 라벨/숫자-단위/범위/기호
        t = _RE_REF_LABEL.sub(" ", t)
        t = _RE_PAREN_NUM.sub(" ", t)
        t = _RE_RANGE.sub(" ", t)
        t = _RE_WT_VOL_MOL.sub(" ", t)
        t = _RE_NUM_UNIT.sub(" ", t)
        t = _RE_SYMBOLS.sub(" ", t)
        t = _RE_CLAIM_KO_INLINE.sub(' ', t)
        t = _RE_DIGIT.sub(' ', t)
        t = _RE_INLINE_FORMULA.sub(' ', t)

        # 빅그램 제거(옵션)
        for pat in _RE_PHRASES:
            t = pat.sub(" ", t)

        # 공백 정리
        t = re.sub(r"\s+", " ", t).strip()

        # 토큰화 후 불용어/숫자/1글자 제거|
        toks = word_tokenize(t)
        toks = [w for w in toks if w not in ALL_STOP and len(w) > 1 and not w.isdigit()]

        out.append(" ".join(toks))
    return out

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
#임베딩 함수
client = OpenAIClient(api_key=open_ai_key)

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=open_ai_key
)

representation_model = OpenAI(
    client = client,
    model="gpt-4.1-mini",
    api_key=open_ai_key,
    verbose=True,
    chat=True
)

In [ ]:
#배치 처리로 문서 임베딩 생성 (tqdm 적용)
def create_embeddings_with_progress(docs, batch_size=100):
    embeddings = []

    # 전체 배치 수 계산
    total_batches = (len(docs) + batch_size - 1) // batch_size

    with tqdm(total=total_batches, desc="Creating embeddings") as pbar:
        for i in range(0, len(docs), batch_size):
            batch = docs[i:i+batch_size]
            batch_embeddings = embedding_model.embed_documents(batch)
            embeddings.extend(batch_embeddings)

            # 진행률 업데이트
            pbar.set_postfix({
                'Processed': f"{min(i+batch_size, len(docs))}/{len(docs)} docs"
            })
            pbar.update(1)

    return np.array(embeddings)

### 함수 정의

In [ ]:
_EXTRA_STOP = ["placeholdertoken"]

# 2) 한글 2자+ 또는 대문자 약어 3자+만 허용(1글자/숫자 단독 컷)
_TOKEN_RE = re.compile(r"[가-힣]{2,}|[A-Z]{3,}(?:-[A-Z0-9]+)*")


def _ko_simple_analyzer(doc: str):
    if not isinstance(doc, str):
        return []
    toks = _TOKEN_RE.findall(doc)
    # 숫자만 / 소문자 1~2자 방지
    out = []
    for t in toks:
        if re.fullmatch(r"\d+(\.\d+)?", t):
            continue
        if re.fullmatch(r"[a-z]{1,2}", t):
            continue
        out.append(t)
    return out

In [ ]:
def _build_vectorizer(min_df: int = 2):
    # 한글/영문/숫자 모두 허용되도록 바운더리 완화
    return CountVectorizer(
        max_df=0.95,
        min_df=2,                     # ← 전달받은 인자 사용
        stop_words=bertopic_stopwords,
        ngram_range=(1, 1),
        token_pattern=r"(?u)[\w\-]+",
        strip_accents='unicode',
        lowercase=True
    )

In [ ]:
#최적의 파라미터 세팅 함수

def rescale_params_for_full_run(
    N: int,
    best,
    umap_used,
    target_K: int = 120,
    mode: str = "balance"  # 유지하되, 이제 동작에는 영향 없음(호환 목적)
):
    """
    샘플 그리드서치에서 고른 UMAP은 '그대로' 유지(freeze).
    풀런 크기 N에 맞춰 HDBSCAN만 스케일링.
    """
    umap_new = dict(umap_used) if umap_used else {}
    umap_new.setdefault("n_neighbors", 60)
    umap_new.setdefault("min_dist", 0.10)
    umap_new.setdefault("n_components", max(umap_new.get("n_components", 10), 10))
    umap_new.setdefault("metric", "cosine")
    umap_new.setdefault("random_state", 42)
    # n_neighbors < n_samples 제약 캡핑
    n = max(1, N)
    umap_new["n_neighbors"] = min(umap_new["n_neighbors"], max(2, n - 1))

    # target_size는 K를 과도하게 늘리지 않도록 하는 '하한' 가드
    target_size = max(40, int(N / max(10, target_K)))
    mcs = int(best.get("min_cluster_size", 50) or 50)
    ms  = int(best.get("min_samples", 10) or 10)
    eps = float(best.get("epsilon", 0.10))

    hdb_new = dict(
        min_cluster_size=mcs,
        min_samples=ms,
        cluster_selection_epsilon=eps,
        cluster_selection_method=best.get("cluster_selection_method", "eom"),
        metric="euclidean",
        prediction_data=False,
    )
    return umap_new, hdb_new

In [ ]:
#평가지표 함수

def _normalize_docs(docs: List[Any]) -> List[str]:
    out = []
    for x in docs:
        if x is None: out.append("")
        elif isinstance(x, str): out.append(x)
        elif isinstance(x, (list, tuple)): out.append(" ".join(map(str, x)))
        else: out.append(str(x))
    return out

def _sanitize_docs_for_vectorizer(docs: List[str]) -> List[str]:
    """완전 공백 문서 방지용 placeholder 처리."""
    out = []
    for d in docs:
        s = (d or "").strip()
        out.append(s if s else "placeholdertoken")
    return out

def _sample_pair(docs: List[str], emb: np.ndarray, k: int = 1000, seed: int = 42):
    rng = np.random.default_rng(seed)
    k = min(k, len(docs))
    idx = rng.choice(len(docs), size=k, replace=False) if k>0 else np.array([], dtype=int)
    return [docs[i] for i in idx], emb[idx], idx

def _safe_entropy(counts: np.ndarray) -> float:
    s = counts.sum()
    if s <= 0: return 0.0
    p = counts / s; p = p[p>0]
    return float(-np.sum(p*np.log(p)))

def _gini(values: List[int]) -> float:
    """List/ndarray 모두 안전. 비어있거나 합이 0이면 0."""
    x = np.asarray(values, dtype=float).ravel()
    n = x.size
    if n == 0:
        return 0.0
    s = x.sum()
    if s <= 0:
        return 0.0
    x.sort()
    cum = np.cumsum(x)
    # 0=균등, 1=극단
    return float((n + 1 - 2 * np.sum(cum) / s) / n)

#outlier_ratio : 노이즈 비율
#gini_clean : 지니계수 토픽분포 불균등도 계산
#top1_clean : 가장 큰 토픽이 차지하는 비율
#n_topics_clean : 실제 토픽 개수(노이즈 제외)
#h_norm : 정규화 엔트로피

def _topic_metrics(info: pd.DataFrame) -> Dict[str, Any]:
    if info is None or info.empty:
        return dict(outlier_ratio=np.nan, gini_clean=np.nan, top1_clean=np.nan,
                    n_topics_clean=0, h_norm=np.nan)

    total = int(info["Count"].sum())
    out_cnt = int(info.loc[info.Topic == -1, "Count"].sum()) if (-1 in info.Topic.values) else 0
    outlier = (out_cnt / total) if total > 0 else np.nan

    clean = info[info.Topic != -1]
    if clean.empty:
        return dict(outlier_ratio=outlier, gini_clean=np.nan, top1_clean=np.nan,
                    n_topics_clean=0, h_norm=np.nan)

    counts = np.asarray(clean["Count"].astype(int).values, dtype=float).ravel()
    counts_sum = counts.sum()
    if counts_sum <= 0:
        return dict(outlier_ratio=outlier, gini_clean=0.0, top1_clean=np.nan,
                    n_topics_clean=int(len(counts)), h_norm=np.nan)

    k = int(len(counts))
    p = counts / counts_sum
    top1 = float(p.max())

    # 정규화 엔트로피
    h = float(-np.sum(p * np.log(p)))
    h_norm = float(h / (math.log(k) if k > 1 else 1.0))

    return dict(outlier_ratio=float(outlier),
                gini_clean=_gini(counts),
                top1_clean=top1,
                n_topics_clean=k,
                h_norm=h_norm)

# “좋을수록 큰 점수”(연속 가중합)
def _score_soft(m: Dict[str, Any]) -> float:
    out = m.get("outlier_ratio");   out = 1.0 - float(out) if (out==out) else 0.5
    hn  = m.get("h_norm");          hn  = float(hn) if (hn==hn) else 0.5
    t1  = m.get("top1_clean");      t1  = float(t1) if (t1==t1) else 0.5
    g   = m.get("gini_clean");      g   = float(g) if (g==g) else 0.5
    top1_term = 1.0 - max(0.0, t1-0.40)/0.60
    top1_term = max(0.0, min(1.0, top1_term))
    g_term = 1.0 - max(0.0, min(1.0, g))     # gini 낮을수록 가점
    return float(0.35*out + 0.25*hn + 0.25*top1_term + 0.15*g_term)


In [ ]:

# ============================ Grid Search ============================
def grid_search_umap_hdbscan_on_sample(
    docs: List[str], emb: np.ndarray,
    sample_size: int = 1000,
    umap_grid: Optional[List[dict]] = None,
    mcs_grid: Optional[List[int]] = None,
    ms_grid: Optional[List[int]]  = None,
    eps_grid: Optional[List[float]] = None,
    min_topic_size: int = 10,
    seed: int = 42,
    # 하드 제약(요청사항)
    max_noise: float = 0.10,
    max_topics: int = 50,
    # 로그
    show_progress: bool = True
) -> Tuple[Dict[str, Any], pd.DataFrame, dict]:
    t0 = time.time()
    # 입력 정리
    assert len(docs) == emb.shape[0], f"docs({len(docs)}) != emb({emb.shape[0]})"
    docs = _normalize_docs(docs)
    emb  = np.asarray(emb, dtype=np.float32)
    emb  = np.nan_to_num(emb, nan=0.0, posinf=0.0, neginf=0.0)

    docs_s, emb_s, _ = _sample_pair(docs, emb, k=sample_size, seed=seed)

    # 소문서 셋 방어: 토큰 드랍 방지용 min_df 동적 조정
    min_df = 1 if max(1, len(docs_s)) < 100 else 2
    vec = _build_vectorizer(min_df=min_df)
    docs_s_vec = _sanitize_docs_for_vectorizer(docs_s)

    n_samp = emb_s.shape[0]

    # 기본 그리드
    if umap_grid is None:
        umap_grid = [
            dict(n_neighbors=30, n_components=10, min_dist=0.15, metric="cosine", random_state=seed),
        ]
    if mcs_grid is None: mcs_grid=[20, 25, 30, 40]
    if ms_grid  is None: ms_grid =[6, 8, 10, 12]
    if eps_grid is None: eps_grid=[0.05, 0.10, 0.20]

    def _cap_umap_params(up: dict, n: int) -> dict:
        up2 = dict(up)
        # n_neighbors must be < n_samples
        if n <= 2:
            up2["n_neighbors"] = max(2, n-1) if n > 1 else 2
        else:
            up2["n_neighbors"] = min(up["n_neighbors"], max(2, n-1))
        return up2

    # 조합 생성 + 진행률 바
    combos = list(product(range(len(umap_grid)), mcs_grid, ms_grid, eps_grid))
    pbar = tqdm(combos, desc="[GridSearch] UMAP×HDBSCAN (1k)", ncols=100, disable=not show_progress)

    rows, objs = [], []
    umap_cache = {}

    for ui, mcs, ms, eps in pbar:
        base_up = umap_grid[ui]
        up = _cap_umap_params(base_up, n_samp)

        # UMAP 캐시
        if ui not in umap_cache:
            try:
                umap_cache[ui] = UMAP(**up).fit_transform(emb_s)
            except Exception as e:
                rows.append(dict(umap_n_neighbors=up.get("n_neighbors"), umap_n_components=up.get("n_components"),
                                 umap_min_dist=up.get("min_dist"), min_cluster_size=None, min_samples=None, epsilon=None,
                                 outlier_ratio=np.nan, gini_clean=np.nan, top1_clean=np.nan, n_topics_clean=0, h_norm=np.nan,
                                 soft_score=-1.0, feas=False, viol=1e9, note=f"umap_error:{type(e).__name__}:{e}"))
                # 해당 ui로 엮인 모든 조합은 스킵
                continue

        red = umap_cache[ui]
        # 진행 문구 업데이트(선택)
        pbar.set_postfix_str(f"UMAP n={up['n_neighbors']}, mcs={mcs}, ms={ms}, eps={eps}")

        try:
            hdb = HDBSCAN(min_cluster_size=mcs, min_samples=ms,
                          cluster_selection_epsilon=eps, metric="euclidean", prediction_data=False)
            tm = BERTopic(embedding_model=None, umap_model=None, hdbscan_model=hdb,
                          min_topic_size=min_topic_size, vectorizer_model=vec, verbose=False)
            _t, _ = tm.fit_transform(docs_s_vec, embeddings=red)
            info = tm.get_topic_info()
            met  = _topic_metrics(info)
            soft = _score_soft(met)

            # 제약 위반량(0이면 Feasible)
            out_v = max(0.0, (met["outlier_ratio"] - max_noise)) if met["outlier_ratio"]==met["outlier_ratio"] else 1.0
            k_v   = max(0.0, (met["n_topics_clean"] - max_topics)) if met["n_topics_clean"]==met["n_topics_clean"] else 10.0
            viol  = 100.0*out_v + 1.0*k_v  # 노이즈 위반 강벌

            rec = dict(
                umap_n_neighbors=up["n_neighbors"], umap_n_components=up["n_components"], umap_min_dist=up["min_dist"],
                min_cluster_size=mcs, min_samples=ms, epsilon=eps,
                **met,
                soft_score=soft,
                feas=(viol==0.0),
                viol=viol,
                umap_params=up, model=tm, info=info
            )
            objs.append(rec)
            rows.append({k:v for k,v in rec.items() if k not in ("model","info","umap_params")})
        except Exception as e:
            rows.append(dict(
                umap_n_neighbors=up["n_neighbors"], umap_n_components=up["n_components"], umap_min_dist=up["min_dist"],
                min_cluster_size=mcs, min_samples=ms, epsilon=eps,
                outlier_ratio=np.nan, gini_clean=np.nan, top1_clean=np.nan, n_topics_clean=0, h_norm=np.nan,
                soft_score=-1.0, feas=False, viol=1e9, note=f"hdb_error:{type(e).__name__}:{e}"
            ))

    df = pd.DataFrame(rows)

    # NaN 채움(정렬 안정화)
    if not df.empty:
        fill_map = {
            "soft_score": -1.0,
            "h_norm": -1.0,
            "n_topics_clean": 10**9,
            "outlier_ratio": np.nan,  # 정렬에는 사용 안 하므로 그대로
        }
        for c, v in fill_map.items():
            if c in df.columns:
                df[c] = df[c].fillna(v)
        if "feas" in df.columns:
            df["feas"] = df["feas"].fillna(False)

    # 랭킹(Feasible → soft_score 내림차순 → h_norm 내림차순 → K 오름차순)
    feasible = df[df["feas"]==True].copy() if not df.empty else pd.DataFrame()
    if not feasible.empty:
        rank = feasible.sort_values(
            ["soft_score","h_norm","n_topics_clean"], ascending=[False,False,True]
        ).reset_index(drop=True)
        top = rank.iloc[0].to_dict()
        # 객체 매칭
        for r in objs:
            if (r["umap_params"]["n_neighbors"]==top["umap_n_neighbors"] and
                r["umap_params"]["n_components"]==top["umap_n_components"] and
                r["umap_params"]["min_dist"]==top["umap_min_dist"] and
                r["min_cluster_size"]==top["min_cluster_size"] and
                r["min_samples"]==top["min_samples"] and
                r["epsilon"]==top["epsilon"]):
                if show_progress:
                    print(f"\n[OK] Feasible found in {time.time()-t0:.1f}s "
                          f"(noise<{max_noise}, K≤{max_topics})")
                return r, rank, r["umap_params"]

    # Feasible 없음 → 위반량 최소(viol 오름차순), 그 다음 soft_score 내림차순
    if not df.empty and len(objs)>0:
        near = df.sort_values(["viol","soft_score","h_norm"], ascending=[True,False,False]).reset_index(drop=True)
        top = near.iloc[0].to_dict()
        for r in objs:
            if (r["umap_params"]["n_neighbors"]==top["umap_n_neighbors"] and
                r["umap_params"]["n_components"]==top["umap_n_components"] and
                r["umap_params"]["min_dist"]==top["umap_min_dist"] and
                r["min_cluster_size"]==top["min_cluster_size"] and
                r["min_samples"]==top["min_samples"] and
                r["epsilon"]==top["epsilon"]):
                if show_progress:
                    print(f"\n[WARN] No feasible combo. Using closest candidate "
                          f"(viol={float(top['viol']):.3f}) in {time.time()-t0:.1f}s")
                return r, near, r["umap_params"]

    # 전부 에러 → 기본값 폴백
    up = dict(n_neighbors=15, n_components=8, min_dist=0.10, metric="cosine", random_state=seed)
    try:
        red = UMAP(**(lambda p: dict(p, **{"n_neighbors": min(max(2, n_samp-1), p["n_neighbors"]) }))(up)).fit_transform(emb_s)
        hdb = HDBSCAN(min_cluster_size=50, min_samples=8, cluster_selection_epsilon=0.10, metric="euclidean", prediction_data=False)
        tm  = BERTopic(embedding_model=None, umap_model=None, hdbscan_model=hdb,
                       min_topic_size=min_topic_size, vectorizer_model=vec, verbose=False)
        _t,_ = tm.fit_transform(_sanitize_docs_for_vectorizer(docs_s), embeddings=red)
        info = tm.get_topic_info(); met = _topic_metrics(info)

        out_v = max(0.0, (met["outlier_ratio"] - max_noise)) if met["outlier_ratio"]==met["outlier_ratio"] else 1.0
        k_v   = max(0.0, (met["n_topics_clean"] - max_topics)) if met["n_topics_clean"]==met["n_topics_clean"] else 10.0
        viol  = 100.0*out_v + 1.0*k_v

        best = dict(umap_params=up, min_cluster_size=50, min_samples=8, epsilon=0.10, model=tm, info=info,
                    **met, soft_score=_score_soft(met), feas=(met["outlier_ratio"]<max_noise and met["n_topics_clean"]<=max_topics),
                    viol=viol, note="fallback")
        rank = pd.DataFrame([{**{k:best.get(k) for k in ("outlier_ratio","n_topics_clean","h_norm","soft_score","feas","viol")},
                              "umap_n_neighbors":up["n_neighbors"], "umap_n_components":up["n_components"],
                              "umap_min_dist":up["min_dist"], "min_cluster_size":50, "min_samples":8, "epsilon":0.10,
                              "note":"fallback"}])
        if show_progress:
            print(f"\n[FALLBACK] All combos failed. Returned default in {time.time()-t0:.1f}s")
        return best, rank, up
    except Exception as e:
        raise RuntimeError(f"[FATAL] Grid+Fallback failed: {e}")


In [ ]:
def run_full_with_best(
    docs: List[str], emb: np.ndarray,
    best: Dict[str,Any], umap_used: dict,
    min_topic_size: int = 10, verbose: bool = False,
    max_noise: float = 0.10, max_topics: int = 50
) -> Dict[str, Any]:
    docs = _normalize_docs(docs)
    docs_vec = _sanitize_docs_for_vectorizer(docs)
    emb  = np.asarray(emb, dtype=np.float32)
    emb  = np.nan_to_num(emb, nan=0.0, posinf=0.0, neginf=0.0)

    # UMAP n_neighbors 제약 캡
    def _cap_umap_params(up: dict, n: int) -> dict:
        up2 = dict(up)
        if n <= 2:
            up2["n_neighbors"] = max(2, n-1) if n > 1 else 2
        else:
            up2["n_neighbors"] = min(up["n_neighbors"], max(2, n-1))
        return up2
    umap_used = _cap_umap_params(umap_used, emb.shape[0])

    # Vectorizer 동적 min_df
    min_df = 1 if max(1, len(docs_vec)) < 100 else 2
    vec  = _build_vectorizer(min_df=min_df)

    N = len(docs)
    umap_adj, hdb_adj = rescale_params_for_full_run(N, best, umap_used)

    if verbose:
        print("[FullRun-Scaled] UMAP:", umap_adj)
        print("[FullRun-Scaled] HDB :", hdb_adj)

    # UMAP fit
    u = UMAP(**umap_adj)
    red = u.fit_transform(emb)

    # HDBSCAN+BERTopic
    hdb = HDBSCAN(**hdb_adj)
    tm  = BERTopic(
        embedding_model=None, umap_model=None, hdbscan_model=hdb,
        min_topic_size=min_topic_size,            # ← 전달 누락 보정
        vectorizer_model=vec, verbose=False
    )
    topics, probs = tm.fit_transform(docs, embeddings=red)
    info = tm.get_topic_info()
    met = _topic_metrics(info)

    feasible = (met["outlier_ratio"] < max_noise) and (met["n_topics_clean"] <= max_topics)
    if verbose:
        print(f"[FullRun] Metrics: noise={float(met['outlier_ratio']):.3f}, "
              f"K={int(met['n_topics_clean'])}, "
              f"H_norm={(met['h_norm'] if met['h_norm']==met['h_norm'] else float('nan')):.3f} "
              f"=> FEASIBLE={feasible}")

    return dict(
        model=tm, umap=u, topics=topics, probs=probs, info=info,
        metrics=met, feasible=feasible,
        used_params=dict(umap=umap_adj,                     # ← 실제 사용값 기록
                         hdb=dict(min_cluster_size=hdb_adj["min_cluster_size"],
                                  min_samples=hdb_adj["min_samples"],
                                  epsilon=hdb_adj["cluster_selection_epsilon"]))
    )

### 메인 파트

In [ ]:
file_path = '/content/압력기반공급제어.xlsx'  # Colab에서는 직접 업로드
df_topic = pd.read_excel(file_path)

In [ ]:
df_topic.columns

Index(['출원번호', '국가코드', '발명의 명칭', '대표청구항', '요약', '기술명', '소주제', '점수', '근거', '순서',
       '번역'],
      dtype='object')

In [ ]:
category=df_topic['기술군'].unique()

In [ ]:
for i in category :

    df_topic_temp=df_topic[df_topic['기술군'] == i]
    df_list = df_topic_temp['번역'].tolist()
    # 불용어 및 전처리
    df_list_pro=simple_preprocess_for_embedding(df_list)

In [ ]:
df_list = df_topic['번역'].tolist()

In [ ]:
# 불용어 및 전처리
df_list_pro=simple_preprocess_for_embedding(df_list)

In [ ]:
embeddings = create_embeddings_with_progress(df_list_pro)
print(f"\n Embeddings shape: {embeddings.shape}")
embeddings = np.array(embeddings)  # 혹시 list이면 array로 변환

Creating embeddings: 100%|██████████| 8/8 [00:17<00:00,  2.22s/it, Processed=726/726 docs]



 Embeddings shape: (726, 1536)


In [ ]:
# embedding 저장/로드하는 코드
# np.save('/content/drive/MyDrive/Patent analysis/ETRI/embeddings_ETRI_양자기술_모집단.npy', embeddings_quantum)
# embeddings_quantum=np.load('/content/drive/MyDrive/Patent analysis/ETRI/embeddings_ETRI_양자기술.npy',mmap_mode = 'r')

In [ ]:
docs = df_list_pro  # List[str]
emb  = embeddings   # np.ndarray (N x d)

# docs: List[str], emb: np.ndarray
best, rank, umap_used = grid_search_umap_hdbscan_on_sample(
    docs, emb, sample_size=200, show_progress=True
)
print(rank.head(10)[[
    "umap_n_neighbors","umap_n_components","umap_min_dist",
    "min_cluster_size","min_samples","epsilon",
    "outlier_ratio","n_topics_clean","h_norm","soft_score","feas","viol"
]])

full = run_full_with_best(docs, emb, best, umap_used, min_topic_size=10, verbose=True)
# full["metrics"], full["feasible"], full["used_params"]

[GridSearch] UMAP×HDBSCAN (1k): 100%|█| 48/48 [00:29<00:00,  1.64it/s, UMAP n=30, mcs=40, ms=12, eps



[OK] Feasible found in 29.3s (noise<0.1, K≤50)
   umap_n_neighbors  umap_n_components  umap_min_dist  min_cluster_size  \
0                30                 10           0.15                20   
1                30                 10           0.15                20   
2                30                 10           0.15                20   
3                30                 10           0.15                25   
4                30                 10           0.15                20   
5                30                 10           0.15                20   
6                30                 10           0.15                20   
7                30                 10           0.15                20   
8                30                 10           0.15                25   
9                30                 10           0.15                25   

   min_samples  epsilon  outlier_ratio  n_topics_clean    h_norm  soft_score  \
0            6     0.05          0.070        

In [ ]:
topic_model = copy.deepcopy(full['model'])

In [ ]:
full['model'].get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,59,-1_expansion_ti_turbine_transfer,"[expansion, ti, turbine, transfer, least, valv...",[an expansion turbine/compressor high-pressure...
1,0,114,0_valve_path_second_side,"[valve, path, second, side, vessel, each, hose...",[provided is valve which can be easily assembl...
2,1,89,1_accumulator_cell_vehicle_purpose,"[accumulator, cell, vehicle, purpose, dispense...",[the purpose of the present invention is to pr...
3,2,68,2_charging_second_transfer_vehicle,"[charging, second, transfer, vehicle, module, ...",[at least two branch transfer of which one end...
4,3,56,3_liquefied_cell_stack_electrochemical,"[liquefied, cell, stack, electrochemical, ship...",[fuel supply system for ship is disclosed acco...
5,4,53,4_value_amount_volume_vehicle,"[value, amount, volume, vehicle, sensor, step,...",[to provide gas filling method in which it is ...
6,5,50,5_accumulator_cylinder_line_refrigerant,"[accumulator, cylinder, line, refrigerant, coo...",[an object of the present invention is to supp...
7,6,39,6_pump_engine_passage_injector,"[pump, engine, passage, injector, vaporizer, o...",[the purpose of the present invention is to su...
8,7,38,7_cell_main_valve_reducing,"[cell, main, valve, reducing, energy, stop, se...",[the present invention relates to fuel cell sy...
9,8,34,8_charging_cooling_freezing_charged,"[charging, cooling, freezing, charged, couplin...",[disclosed is an for preventing hydrogen charg...


In [ ]:
topic_model.get_topic_info().to_excel(f'./topic_result_AI요약.xlsx')

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
df_topic['topic']=topic_model.topics_
df_topic.to_excel(f'./압력기반공급제어_토픽.xlsx',index=False)

In [ ]:
# Method 3 - pickle
full['model'].save("/content/drive/MyDrive/Patent analysis/ETRI/Model_AI요약", serialization="pickle")

2025-10-04 14:39:41,205 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
df_topic